# Evaluation of Lyft-Tracks

In [1]:
## set vars
LYFT_DATASET_ROOT = '/home/itiv/Desktop/lyft-kaggle-dataset/train/'
CONFIG_DIR = "./Lyft-Detector/second.pytorch/second/configs/nuscenes/"
MODEL_DIR = './Lyft-Detector/second.pytorch/second/model/'
#MODEL = 'small' #small,middle,large
VERSION = "v1.0-trainval" #v1.0-test for test-dataset

In [2]:
#### load ground truth data
# Load the SDK
%matplotlib inline
from lyft_dataset_sdk.lyftdataset import LyftDataset

# Load the dataset
# Adjust the dataroot parameter below to point to your local dataset path.
# The correct dataset path contains at least the following four folders (or similar): images, lidar, maps, v1.0.1-train
level5data = LyftDataset(data_path=LYFT_DATASET_ROOT, json_path=LYFT_DATASET_ROOT+'/data', verbose=True)


9 category,
18 attribute,
4 visibility,
18421 instance,
10 sensor,
148 calibrated_sensor,
177789 ego_pose,
180 log,
180 scene,
22680 sample,
189504 sample_data,
638179 sample_annotation,
1 map,
Done loading in 7.9 seconds.
Reverse indexing ...
Done reverse indexing in 2.2 seconds.


In [3]:
### create pkl files
def create_pkl(scene_num):
    scene = level5data.scene[scene_num]
    scene_token = scene['token']
    version = "v1.0-trainval"
    !python Lyft-Detector/second.pytorch/second/create_data.py nuscenes_data_prep --root_path=$LYFT_DATASET_ROOT  --version=$version --dataset_name="NuScenesDataset" --scene_token=$scene_token --max_sweeps=10

def set_config_file(scene_num,model):
    import numpy as np
    from second.protos import pipeline_pb2
    from google.protobuf import text_format
    scene = level5data.scene[scene_num]
    scene_token = scene['token']
    config_path = CONFIG_DIR+'/all.pp.lowa_'+model+'_range.config'
    info_path = LYFT_DATASET_ROOT+'/infos_'+scene_token+'.pkl'

    config = pipeline_pb2.TrainEvalPipelineConfig()
    with open(config_path, "r") as f:
        proto_str = f.read()
        text_format.Merge(proto_str, config)
    config.train_input_reader.dataset.kitti_info_path = info_path
    config.train_input_reader.dataset.kitti_root_path = LYFT_DATASET_ROOT

    config.train_input_reader.preprocess.database_sampler.database_info_path = info_path

    config.eval_input_reader.dataset.kitti_info_path = info_path
    config.eval_input_reader.dataset.kitti_root_path = LYFT_DATASET_ROOT

    config_text = text_format.MessageToString(config)
    with open(config_path, "w") as f:
        f.write(config_text)

    print('Paths set in config file')
def track(scene_num,model,net):
    #model: small,middle,range
    scene = level5data.scene[scene_num]
    scene_token = scene['token']
    config_path = CONFIG_DIR+'/all.pp.lowa_'+model+'_range.config'
    ckpt_path= MODEL_DIR+net
    #if model == 'small':
    #    ckpt_path= MODEL_DIR+'/model_standard/voxelnet-48698.tckpt'
    #elif model == 'middle':
    #    ckpt_path = MODEL_DIR+'/model_middle_range/voxelnet-374148.tckpt'
    #else:
    #    ckpt_path = ''

    info_path = LYFT_DATASET_ROOT+'/infos_'+scene_token+'.pkl'
    root_path = LYFT_DATASET_ROOT
    result_path = LYFT_DATASET_ROOT
    version = "v1.0-trainval"

    print('Detecting...this might take several minutes...')
    !python ./Lyft-Detector/second.pytorch/second/detect.py detect --scene_token=$scene_token --config_path=$config_path --ckpt_path=$ckpt_path --info_path=$info_path --root_path=$root_path --result_path=$result_path
    print('Tracking...this might take several minutes...')
    detection_file = result_path+'/detections_'+scene_token+'.json'
    !python ./Lyft-Tracker/mahalanobis_3d_multi_object_tracking/main.py track --save_root=$root_path --version=$version --detection_file=$detection_file --data_root=$root_path
    
    ## generate output
    import json
    import pandas as pd
    from pyquaternion import Quaternion
    import numpy as np
    import math
    import operator
    from operator import add
    from scipy.stats import beta
    from collections import Counter
    import os
    import random

    scene = level5data.scene[scene_num]
    scene_token = scene['token']

    tracking_file = LYFT_DATASET_ROOT+'/tracking_results_'+scene_token+'.json'

    with open(tracking_file) as json_file:
        data = json.load(json_file)

    counter = 0

    tracking_ids = []
    sample_token = scene['first_sample_token']
    timestamp = 0
    for index in range(scene['nbr_samples']):
        sample = level5data.get('sample', sample_token)
        for detection_dict in data['results'][sample_token]:
            if not int(detection_dict['tracking_id']) in tracking_ids:
                tracking_ids.append(int(detection_dict['tracking_id']))
            else:
                counter += 1
        sample_token = sample['next']
        if sample_token is '':
            break

    tracking_ids.sort()            

    print(f'Extracted {len(tracking_ids)} tracks')


    index = range(scene['nbr_samples'])
    columns = ["Car "+str(x) for x in tracking_ids]


    traffic_coords = pd.DataFrame(index=index, columns=columns)
    traffic_coords = traffic_coords.astype(object)

    traffic_orientation = pd.DataFrame(index=index, columns=columns)
    traffic_orientation = traffic_orientation.astype(object)

    traffic_sizes = pd.DataFrame(index=index, columns=columns)
    traffic_sizes = traffic_sizes.astype(object)

    columns = ['Timestamp','Translation','Yaw']
    ego_info = pd.DataFrame(index=index, columns=columns)
    ego_info = ego_info.astype(object)


    translation_offset = None

    sample_token = scene['first_sample_token']
    for index in range(scene['nbr_samples']):
        sample = level5data.get('sample', sample_token)
        sample_data = level5data.get('sample_data', sample['data']['LIDAR_TOP'])

        ego_pose = level5data.get('ego_pose', sample_data['ego_pose_token'])

        if translation_offset is None:
            translation_offset = ego_pose['translation']

        ego_info.at[index, 'Timestamp'] = ego_pose['timestamp']
        ego_info.at[index, 'Yaw'] = Quaternion(ego_pose['rotation']).yaw_pitch_roll[0]
        ego_info.at[index, 'Translation'] = list(map(operator.sub,ego_pose['translation'],translation_offset))
        #filter pedestrians and bicycles    
        for detection_dict in data['results'][sample_token]:
            if not (detection_dict['tracking_name'] == 'bicycle' or detection_dict['tracking_name'] == 'pedestrian'):
                traffic_coords.at[index, 'Car '+detection_dict['tracking_id']] =  list(map(operator.sub,detection_dict['translation'],translation_offset))
                traffic_orientation.at[index, 'Car '+detection_dict['tracking_id']] = Quaternion(detection_dict['rotation']).yaw_pitch_roll[0]
                traffic_sizes.at[index, 'Car '+detection_dict['tracking_id']] = detection_dict['size']
        sample_token = sample['next']
        if sample_token is '':
            break

    #filter short occurence traffic <1s
    for index,entrys in traffic_coords.count().iteritems():
        if entrys < 5:
            traffic_coords.drop(index,axis=1,inplace=True)
            traffic_sizes.drop(index,axis=1,inplace=True)
            traffic_orientation.drop(index,axis=1,inplace=True)


    def dist(l1,l2):
        d = math.sqrt((l1[0]-l2[0])**2+(l1[1]-l2[1])**2+(l1[2]-l2[2])**2)
        return d

    #filter intersection traffic with ego
    for (columnName, columnData) in traffic_coords.iteritems():
        for index,item in enumerate(traffic_coords[columnName]):
            if type(ego_info.at[index, 'Translation']) is list and type(item) is list and dist(item,ego_info.at[index, 'Translation']) < 1:
                traffic_coords.drop(columnName,axis=1,inplace=True)
                traffic_sizes.drop(columnName,axis=1,inplace=True)
                traffic_orientation.drop(columnName,axis=1,inplace=True)
                print(f'dropped {columnName}')
                break;


    #interpolate short NaNs
    for (columnName, columnData) in traffic_coords.iteritems():
        for index,item in enumerate(traffic_coords[columnName]):
            if type(item) is list and len(traffic_coords[columnName])>index+1 and type(traffic_coords[columnName][index+1]) is not list:
                for step in range(10):
                    if len(traffic_coords[columnName])>index+1+step and type(traffic_coords[columnName][index+1+step]) is list:
                        v = list(map(operator.sub, traffic_coords[columnName][index+1+step],item))
                        traffic_coords[columnName][index+1] = list(map(operator.add, item, [x / (step+1) for x in v]))
                        traffic_orientation[columnName][index+1] = (traffic_orientation[columnName][index+1+step]-traffic_orientation[columnName][index])/(step+1)
                        traffic_sizes[columnName][index+1] = traffic_sizes[columnName][index]
                        #print(f'{item} {traffic_coords[columnName][index+1]} {traffic_coords[columnName][index+1+step]}')
                        break





    def plotVal(x):
        import numpy as np
        import pandas as pd
        import seaborn as sns
        import matplotlib.pyplot as plt
        from scipy import stats
        plt.figure(figsize=(16, 6))
        sns.distplot(x);


    #suppress random orientation changes 
    #driving vehicle: orientation according to direction
    #standing vehicle: orientation according to most detected

    dt = 0.2 #Periodendauer
    for (columnName, columnData) in traffic_orientation.iteritems():

        traffic_speed = []
        moving = False
        prev_speed = None
        for index,item in enumerate(traffic_coords[columnName]):
            
            if type(item) is list and len(traffic_coords[columnName])>index+1 and type(traffic_coords[columnName][index+1]) is list:
                traffic_speed.append(abs(dist(item,traffic_coords[columnName][index+1])/dt))
                prev_speed = abs(dist(item,traffic_coords[columnName][index+1])/dt)
            elif prev_speed is not None:
                traffic_speed.append(prev_speed)
                prev_speed = None
            else:
                traffic_speed.append(np.nan)

        prev_orientation = None
        not_moving_orientations = []
        not_moving_start_index = 0
        
        for index,item in enumerate(traffic_speed):

            if moving and (abs(item) < 1.5 or math.isnan(item)):
                moving = False
                not_moving_orientations = []
                not_moving_start_index = index
            elif not moving and (abs(item) > 2 or math.isnan(item)): 
                if len(not_moving_orientations) > 1:
                    orientation,_ = (Counter(not_moving_orientations).most_common(1)[0])
                    if not math.isnan(orientation):
                        for idx in range(index-not_moving_start_index):
                            traffic_orientation[columnName][index-idx] = orientation
                moving = True
                
                
            if not math.isnan(item):
                if moving and index >= len(traffic_speed)-5:#ende der liste
                    if prev_orientation is not None:
                        traffic_orientation[columnName][index] = prev_orientation
                elif moving and type(traffic_coords[columnName][index]) is list and type(traffic_coords[columnName][index+5]) is list:
                    v = list(map(operator.sub,traffic_coords[columnName][index],traffic_coords[columnName][index+5]))
                    yaw = math.atan2(v[1],v[0])-np.pi
                    traffic_orientation[columnName][index] = yaw #---> TODO!!!!!!!!!!!!!!!!!!!!
                elif moving and type(traffic_coords[columnName][index]) is list and type(traffic_coords[columnName][index+1]) is not list and prev_orientation is not None:
                    traffic_orientation[columnName][index] = prev_orientation
                elif not moving:
                    not_moving_orientations.append(round(traffic_orientation[columnName][index],1))

                prev_orientation = traffic_orientation[columnName][index]
            else:
                traffic_orientation[columnName][index] = prev_orientation
                
        if len(not_moving_orientations) > 1:
            orientation,_ = (Counter(not_moving_orientations).most_common(1)[0])
            if not math.isnan(orientation):
                for idx in range(len(traffic_speed)-not_moving_start_index-1):
                    traffic_orientation[columnName][index-idx] = orientation

    ## create result_json_dict #####
    
    
    print(traffic_orientation)
    print(traffic_coords)


    gt_dict = {'samples':[]}

    #iterate over rows
    for index, row in traffic_coords.iterrows():
        #iterate over columns
        sample_dict = {'token':'frame_'+str(index),'timestamp':index*0.2,'anns':[]}

        for colIndex in range(len(row)):
            if type(traffic_coords.iloc[index,colIndex]) is list:
                ann_dict = {'instance':colIndex,'translation':traffic_coords.iloc[index,colIndex],'size':traffic_sizes.iloc[index,colIndex],'rotation':traffic_orientation.iloc[index,colIndex]}
                sample_dict['anns'].append(ann_dict)
        gt_dict['samples'].append(sample_dict)


    def dist(l1,l2):
        d = math.sqrt((l1[0]-l2[0])**2+(l1[1]-l2[1])**2+(l1[2]-l2[2])**2)
        return d
    
    ##########

    CM_PROJECT_DIR =  LYFT_DATASET_ROOT+'/CarMaker'#'C://CM_Projects//maneuver_simu'
    TESTRUN_DIR = CM_PROJECT_DIR+'/TestRun'#CM_PROJECT_DIR+'//Data//TestRun'
    SIMINPUT_DIR = CM_PROJECT_DIR+'/SimInput/'#CM_PROJECT_DIR+'//SimInput'

    GROUND_TRUTH_DIR = CM_PROJECT_DIR+'/gt_data/'  #only for eval

    if not os.path.exists(CM_PROJECT_DIR):
        os.mkdir(CM_PROJECT_DIR)
    if not os.path.exists(TESTRUN_DIR):
        os.mkdir(TESTRUN_DIR)
    if not os.path.exists(SIMINPUT_DIR):
        os.mkdir(SIMINPUT_DIR)
    if not os.path.exists(GROUND_TRUTH_DIR):
        os.mkdir(GROUND_TRUTH_DIR)

    SCENE_NUM = scene_num

    TESTRUN_ID = 'scene-'+str(SCENE_NUM)+'-'+model
    TESTRUN_NAME = f'//testrun_{TESTRUN_ID}'
    TESTRUN_PATH = TESTRUN_DIR+TESTRUN_NAME
    TRAFFIC_PROFILE_NAME = f'traffic_profile_{TESTRUN_ID}.txt'
    TRAFFIC_PROFILE_PATH = SIMINPUT_DIR+TRAFFIC_PROFILE_NAME
    EGO_PROFILE_NAME = f'ego_profile_{TESTRUN_ID}.txt'
    EGO_PROFILE_PATH = SIMINPUT_DIR+EGO_PROFILE_NAME

    gt_data_name = f'gt_data_{TESTRUN_ID}.json'
    with open(GROUND_TRUTH_DIR+gt_data_name, 'w') as fp:
        print('OPENED '+GROUND_TRUTH_DIR+gt_data_name)
        json.dump(gt_dict, fp)



    content_lines = []
    content_lines.append('#INFOFILE1.1 - Do not remove this line!\n\
FileIdent = CarMaker-TestRun 8\n\
FileCreator = CarMaker 8.1 2019-11-07\n\
Description:\n\
Vehicle = UserVehicle_MyCar\n\
Trailer =\n\
Tire.0 =\n\
Tire.1 =\n\
Tire.2 =\n\
Tire.3 =\n\
Snapshot.TimeLimit =\n\
Snapshot.DistLimit =\n\
VehicleLoad.0.mass = 0\n\
VehicleLoad.0.pos = 0 0 0\n\
VehicleLoad.1.mass = 0\n\
VehicleLoad.1.pos = 0 0 0\n\
VehicleLoad.2.mass = 0\n\
VehicleLoad.2.pos = 0 0 0\n\
VehicleLoad.3.mass = 0\n\
VehicleLoad.3.pos = 0 0 0\n\
TrailerLoad.0.mass = 0\n\
TrailerLoad.0.pos = 0 0 0\n\
TrailerLoad.1.mass = 0\n\
TrailerLoad.1.pos = 0 0 0\n\
TrailerLoad.2.mass = 0\n\
TrailerLoad.2.pos = 0 0 0\n')

    #ego maneuver for every 10th coord

    time_between_records = 0.2 

    start_vel = 0#ego_vel[0]
    man_counter = 1
    #ego_man = []
    #for vel in ego_vel[::10]:
    #    ego_man.append('DrivMan.'+str(man_counter)+'.TimeLimit = '+str(10*time_between_records)+'\n\
    #DrivMan.'+str(man_counter)+'.LongDyn = VelControl '+str(vel*3.6)+' 0.0 1.0 0 1 0\n\
    #DrivMan.'+str(man_counter)+'.LatDyn = Driver 0\n')
    #    man_counter += 1

           # +str(start_vel)+
    content_lines.append('DrivMan.Init.Velocity = 0\n\
DrivMan.Init.SteerAng = 0\n\
DrivMan.Init.GearNo = 0\n\
DrivMan.Init.LaneOffset = 0\n\
DrivMan.Init.OperatorActive = 1\n\
DrivMan.Init.OperatorState = drive\n\
DrivMan.VhclOperator.Kind = IPGOperator 1\n\
DrivMan.nDMan = '+str(man_counter)+'\n')

    content_lines.append('DrivMan.0.TimeLimit = '+str(time_between_records*(len(ego_info)-1))+'\n\
DrivMan.0.LongDyn = Stop 2.0 0\n\
DrivMan.0.LatDyn = Driver 0\n')


    #write traffic profile file

    traffic_profile = []
    traffic_profile.append('# Time')
    for index,col in enumerate(traffic_coords):
        traffic_profile.append(' FM_tx_%d FM_ty_%d FM_tz_%d FM_rz_%d' % (index,index,index,index))
    traffic_profile.append('\n')    


    prev_row = None
    for time_index, row in traffic_coords.iterrows():
        traffic_profile.append('%f'%(time_index*time_between_records))
        for car_index,coords in enumerate(row):
            if type(coords) is list:
                if(time_index == None or car_index == None):
                    print('index error')
                    traffic_profile.append(' %f %f %f %f'%(0,0,-100,0))
                elif(traffic_orientation.iloc[time_index,car_index] == None):
                    print('#####orientation error!!!')
                    traffic_profile.append(' %f %f %f %f'%(0,0,-100,0))
                else:
                    traffic_profile.append(' %f %f %f %f'%(coords[0],coords[1],0.6,traffic_orientation.iloc[time_index,car_index]))
            else:
                traffic_profile.append(' %f %f %f %f'%(0,0,-100,0))

                
        prev_row = row
        traffic_profile.append('\n') 

    with open(TRAFFIC_PROFILE_PATH, 'w') as f:
        for item in traffic_profile:
            f.write("%s" % item)


    traffic_counter = 0   
    traffic_counter = len(traffic_coords.columns)
    content_lines.append('Traffic.IFF.FName = SimInput/'+TRAFFIC_PROFILE_NAME+'\n\
Traffic.IFF.Time.Name = Time\n\
Traffic.N = %d\n\
Traffic.SpeedUnit = ms\n'%traffic_counter)


    #get average sizes of vehicle

    traffic_sizes_avr = []
    for (columnName, columnData) in traffic_sizes.iteritems():
        counter = 0
        l_sum = 0
        for size in columnData.values:
            if isinstance(size, list):
                counter += 1
                l_sum += size[1]
        traffic_sizes_avr.append(l_sum/counter)

    small_car = ['Audi_TT_2015.mobj','Citroen_C3_2015.mobj','Honda_Fit_2015.mobj'] #l<4m
    medium_car = ['Audi_A4AllRoad_2016.mobj','Audi_A7_2018.mobj','BMW_5_2017.mobj','Honda_CivicTypeR_2018.mobj','MB_AClass_2018.mobj','MB_CClass_2015.mobj','Jaguar_FType_2017.mobj'] #l<5m
    large_car = ['Chevrolet_Silverado1500_2013.mobj','Chrysler_Pacifica_2016.mobj','Dodge_GrandCaravan_2011.mobj','LandRover_RangeRover_2014.mobj','MB_XClass_2018.mobj'] #l<6m
    van = ['Ford_Transit_2014.mobj','MB_Sprinter_2013.mobj','MB_Vito_2014.mobj','VW_T6_2016.mobj','VW_Transporter_2016.mobj'] #l<10m
    bus_truck = ['Coach.mobj','Euro_ConcreteMixer.mobj','Iveco_EurotechLN2_1992.mobj','MAN_TGS_2012.mobj','MB_Actros_1996.mobj','MB_Atego_2013_Move.mobj'] #l>10m



    for index,col in enumerate(traffic_coords): 

        vehicle_path = '3D/Vehicles/'

        if traffic_sizes_avr[index]<4:
            vehicle_path = vehicle_path+small_car[random.randrange(0,len(small_car),1)]
        elif traffic_sizes_avr[index]<5:
            vehicle_path = vehicle_path+medium_car[random.randrange(0,len(medium_car),1)]
        elif traffic_sizes_avr[index]<6:
            vehicle_path = vehicle_path+large_car[random.randrange(0,len(large_car),1)]
        elif traffic_sizes_avr[index]<10:
            vehicle_path = vehicle_path+van[random.randrange(0,len(van),1)]   
        elif traffic_sizes_avr[index]>10:
            vehicle_path = vehicle_path+bus_truck[random.randrange(0,len(bus_truck),1)]   

        content_lines.append('Traffic.'+str(index)+'.ObjectKind = Movable\n\
Traffic.'+str(index)+'.ObjectClass = Car\n\
Traffic.'+str(index)+'.Name = T'+str(index)+'\n\
Traffic.'+str(index)+'.Info = UNNAMED Object '+str(index)+'\n\
Traffic.'+str(index)+'.Movie.Geometry = '+vehicle_path+'\n\
Traffic.'+str(index)+'.Color = 1.0 0.0 0.0\n\
Traffic.'+str(index)+'.Basics.Dimension = 4.28 1.82 1.28\n\
Traffic.'+str(index)+'.Basics.Offset = 0.0 0.0\n\
Traffic.'+str(index)+'.Basics.Fr12CoM = 2.15\n\
Traffic.'+str(index)+'.Init.Orientation = 0.0 0.0 0.0\n\
Traffic.'+str(index)+'.RCSClass = RCS_Car\n\
Traffic.'+str(index)+'.DetectMask = 1 1\n\
Traffic.'+str(index)+'.Route = 0 1\n\
Traffic.'+str(index)+'.Init.Road = 18 R2\n\
Traffic.'+str(index)+'.Init.v = 1\n\
Traffic.'+str(index)+'.FreeMotion = 1\n\
Traffic.'+str(index)+'.UpdRate = 200\n\
Traffic.'+str(index)+'.IFF.FM_tx.Name =FM_tx_'+str(index)+'\n\
Traffic.'+str(index)+'.IFF.FM_tx.Factor = 1.0\n\
Traffic.'+str(index)+'.IFF.FM_tx.Offset = 0.0\n\
Traffic.'+str(index)+'.IFF.FM_ty.Name =FM_ty_'+str(index)+'\n\
Traffic.'+str(index)+'.IFF.FM_ty.Factor = 1.0\n\
Traffic.'+str(index)+'.IFF.FM_ty.Offset = 0.0\n\
Traffic.'+str(index)+'.IFF.FM_tz.Name =FM_tz_'+str(index)+'\n\
Traffic.'+str(index)+'.IFF.FM_tz.Factor = 1.0\n\
Traffic.'+str(index)+'.IFF.FM_tz.Offset = 0.0\n\
Traffic.'+str(index)+'.IFF.FM_rx.Name =\n\
Traffic.'+str(index)+'.IFF.FM_rx.Factor = 1.0\n\
Traffic.'+str(index)+'.IFF.FM_rx.Offset = 0.0\n\
Traffic.'+str(index)+'.IFF.FM_ry.Name =\n\
Traffic.'+str(index)+'.IFF.FM_ry.Factor = 1.0\n\
Traffic.'+str(index)+'.IFF.FM_ry.Offset = 0.0\n\
Traffic.'+str(index)+'.IFF.FM_rz.Name =FM_rz_'+str(index)+'\n\
Traffic.'+str(index)+'.IFF.FM_rz.Factor = 1.0\n\
Traffic.'+str(index)+'.IFF.FM_rz.Offset = 0.0\n')


    ego_coords = ego_info['Translation'].tolist()





    road_points = []

    prev_row = None
    distance = 5.5
    for index,row in enumerate(ego_coords[::1]):  
        if prev_row is None or abs(dist([row[0],row[1],0],prev_row))>1 :
            dx = -1*math.sin(ego_info.at[index, 'Yaw'])*distance
            dy = math.cos(ego_info.at[index, 'Yaw'])*distance
            road_points.append([row[0]+dx,row[1]+dy])
            prev_row=[row[0],row[1],0]



    #pointlist start degree
    dx = road_points[1][0]-road_points[0][0]
    dy = road_points[1][1]-road_points[0][1]

    startDeg = math.degrees(math.atan2(dx, dy))


    #pointlist end vector
    dx = ego_coords[-1][0]-ego_coords[-2][0]
    dy = ego_coords[-1][1]-ego_coords[-2][1]

    content_lines.append('DrivMan.OW.Active = 1\n\
DrivMan.OW.Quantities = Time User1 User2 User3 User4\n\
DrivMan.OW.StartGearNo = 1\n\
DrivMan.OW.StartVelocity =\n\
DrivMan.OW.GasMax = 0.5\n\
DrivMan.OW.RefCh = Time\n\
DrivMan.OW.ConsiderRoadSigns = 0\n\
DrivMan.OW.sRoute.Offset = 0\n\
DrivMan.OW.Time.Name = t[s]\n\
DrivMan.OW.Time.Factor = 1.0\n\
DrivMan.OW.User1.Name = x\n\
DrivMan.OW.User1.Factor = 1.0\n\
DrivMan.OW.User1.Offset = 0.0\n\
DrivMan.OW.User2.Name = y\n\
DrivMan.OW.User2.Factor = 1.0\n\
DrivMan.OW.User2.Offset = 0.0\n\
DrivMan.OW.User3.Name = yaw\n\
DrivMan.OW.User3.Factor = 1.0\n\
DrivMan.OW.User3.Offset = 0.0\n\
DrivMan.OW.User4.Name = -\n\
DrivMan.OW.User4.Factor = 1.0\n\
DrivMan.OW.User4.Offset = 0.0\n\
DrivMan.OW.FName = '+EGO_PROFILE_NAME+'\n\
ErrorClass.0.Action = abort\n\
ErrorClass.0.Save = 0\n\
ErrorClass.0.WarningLimit = 3 5\n\
ErrorClass.1.Action = abort\n\
ErrorClass.1.Save = 0\n\
ErrorClass.1.WarningLimit = 3 5\n\
ErrorClass.2.Action = abort\n\
ErrorClass.2.Save = 0\n\
ErrorClass.2.WarningLimit = 3 5\n\
ErrorClass.3.Action = abort\n\
ErrorClass.3.Save = 0\n\
ErrorClass.3.WarningLimit = 3 5\n\
ErrorClass.4.Action = abort\n\
ErrorClass.4.Save = 0\n\
ErrorClass.4.WarningLimit = 3 5\n\
ErrorClass.5.Action = abort\n\
ErrorClass.5.Save = 0\n\
ErrorClass.5.WarningLimit = 3 5\n\
ErrorClass.6.Action = abort\n\
ErrorClass.6.Save = 0\n\
ErrorClass.6.WarningLimit = 10 5\n\
ErrorClass.7.Action = abort\n\
ErrorClass.7.Save = 0\n\
ErrorClass.7.WarningLimit = 3 5\n\
ErrorClass.10.Action = abort\n\
ErrorClass.10.Save = 0\n\
ErrorClass.10.WarningLimit = 3 5\n\
ErrorClass.11.Action = abort\n\
ErrorClass.11.Save = 0\n\
ErrorClass.11.WarningLimit = 3 5\n\
Road.FileIdent = IPGRoad 8.0\n\
Road.LibVersion = 8.1\n\
Road.Country = DEU\n\
Road.nLinks = 1\n\
Road.nJunctions = 0\n\
Road.nObjects = 149\n\
Road.nRoutes = 1\n\
Road.RoadNetworkLength = 595.296984550432\n\
Road.BBox = -19.5668198180623 547.415304391587 -224.837648089968 34.6179698226834 -11 11\n\
Road.Route.0.Length = 595.296984550432\n\
Road.RST.Unit = kmh\n\
Road.RST = 50 100 130 30 70 30 0 -1\n\
Road.Movie = 0.2 1 0.01 1.5 1.5 1 1\n\
Road.PathMode = -1\n\
Road.Link.0.ID = 0\n\
Road.Link.0.Junctions = -1 -1 -2 -1\n\
Road.Link.0.Node0 = 0 0 0 '+str(startDeg)+'\n\
Road.Link.0.RST = countryroad\n\
Road.Link.0.RL.ID = 1\n\
Road.Link.0.Seg.0.ID = 5\n\
Road.Link.0.Seg.0.Type = PointList\n\
Road.Link.0.Seg.0.Param =  '+str(dx)+' '+str(dy)+' 1 0 0 0 0 0\n\
Road.Link.0.Seg.0.PointList:\n')

    #append road pointlist
    obj_id=5
    pointlist = []
    pointlist.append('# t[s] x y yaw -\n')
    counter = 0
    for index,row in enumerate(ego_coords[::1]):
        pointlist.append('  %f %f %f %f %f\n'%(counter*time_between_records,row[0],row[1],ego_info.at[index, 'Yaw'],0))
        counter += 1


    with open(EGO_PROFILE_PATH, 'w') as f:
        for item in pointlist:
            f.write("%s" % item)


    pointlist = ''



    ##extending road
    d0 = 500 #road extension distance


    # at the end
    #m = (road_points[-1][1]-road_points[-2][1])/(road_points[-1][0]-road_points[-2][0])
    dx = road_points[-2][0]-road_points[-1][0]#dx = d0/math.sqrt(1+m**2)
    dy = road_points[-2][1]-road_points[-1][1]#dy = m*dx
    road_points.append([road_points[-1][0]-dx*100,road_points[-1][1]-dy*100])

    # at the beginning
    #m = (road_points[0][1]-road_points[1][1])/(road_points[0][0]-road_points[1][0])
    dx = road_points[0][0]-road_points[1][0]#d0/math.sqrt(1+m**2)
    dy = road_points[0][1]-road_points[1][1]#m*dx
    #road_points = [[road_points[0][0]+dx*100,road_points[0][1]+dy*100]] + road_points

    for row in road_points:
        obj_id +=  1
        pointlist = pointlist + '	'+str(row[0])+' '+str(row[1])+'\n'

    content_lines.append(pointlist)    

    content_lines.append('Road.Link.0.LaneSection.0.ID = '+str(obj_id+1)+'\n\
Road.Link.0.LaneSection.0.Start = 0\n\
Road.Link.0.LaneSection.0.LaneL.0.ID = '+str(obj_id+2)+'\n\
Road.Link.0.LaneSection.0.LaneL.0 = 0 3.5 3.5 0 0 0 0\n\
Road.Link.0.LaneSection.0.LaneL.0.ARP = '+str(obj_id+5)+' '+str(obj_id+6)+' '+str(obj_id+7)+' '+str(obj_id+8)+' '+str(obj_id+9)+' '+str(obj_id+10)+'\n\
Road.Link.0.LaneSection.0.LaneL.1.ID = '+str(obj_id+13)+'\n\
Road.Link.0.LaneSection.0.LaneL.1 = 0 0.75 0.75 4 0 0 0\n\
Road.Link.0.LaneSection.0.LaneL.1.ARP = '+str(obj_id+16)+' '+str(obj_id+17)+' '+str(obj_id+18)+' '+str(obj_id+19)+' '+str(obj_id+20)+' '+str(obj_id+21)+'\n\
Road.Link.0.LaneSection.0.LaneL.2.ID = '+str(obj_id+24)+'\n\
Road.Link.0.LaneSection.0.LaneL.2 = 0 1.5 1.5 4 0 0 0\n\
Road.Link.0.LaneSection.0.LaneL.2.ARP = '+str(obj_id+27)+' '+str(obj_id+28)+' '+str(obj_id+29)+' '+str(obj_id+30)+' '+str(obj_id+31)+' '+str(obj_id+32)+'\n\
Road.Link.0.LaneSection.0.LaneL.3.ID = '+str(obj_id+34)+'\n\
Road.Link.0.LaneSection.0.LaneL.3 = 0 1.5 1.5 4 0 0 0\n\
Road.Link.0.LaneSection.0.LaneL.3.ARP = '+str(obj_id+37)+' '+str(obj_id+38)+' '+str(obj_id+39)+' '+str(obj_id+40)+' '+str(obj_id+41)+' '+str(obj_id+51)+'\n\
Road.Link.0.LaneSection.0.LaneL.4.ID = '+str(obj_id+44)+'\n\
Road.Link.0.LaneSection.0.LaneL.4 = 0 0.75 0.75 4 0 0 0\n\
Road.Link.0.LaneSection.0.LaneL.4.ARP = '+str(obj_id+47)+' '+str(obj_id+48)+' '+str(obj_id+49)+' '+str(obj_id+50)+' '+str(obj_id+51)+' '+str(obj_id+52)+'\n\
Road.Link.0.LaneSection.0.LaneL.5.ID = '+str(obj_id+55)+'\n\
Road.Link.0.LaneSection.0.LaneL.5 = 0 3.5 3.5 0 0 0 0\n\
Road.Link.0.LaneSection.0.LaneL.5.ARP = '+str(obj_id+58)+' '+str(obj_id+59)+' '+str(obj_id+60)+' '+str(obj_id+61)+' '+str(obj_id+62)+' '+str(obj_id+63)+'\n\
Road.Link.0.LaneSection.0.LaneL.6.ID = '+str(obj_id+66)+'\n\
Road.Link.0.LaneSection.0.LaneL.6 = 0 3.5 3.5 0 0 0 0\n\
Road.Link.0.LaneSection.0.LaneL.6.ARP = '+str(obj_id+69)+' '+str(obj_id+70)+' '+str(obj_id+71)+' '+str(obj_id+72)+' '+str(obj_id+73)+' '+str(obj_id+74)+'\n\
Road.Link.0.LaneSection.0.LaneL.7.ID = '+str(obj_id+77)+'\n\
Road.Link.0.LaneSection.0.LaneL.7 = 0 3.5 3.5 0 0 0 0\n\
Road.Link.0.LaneSection.0.LaneL.7.ARP = '+str(obj_id+80)+' '+str(obj_id+81)+' '+str(obj_id+82)+' '+str(obj_id+83)+' '+str(obj_id+84)+' '+str(obj_id+85)+'\n\
Road.Link.0.LaneSection.0.LaneL.8.ID = '+str(obj_id+88)+'\n\
Road.Link.0.LaneSection.0.LaneL.8 = 0 1 1 4 0 0 0\n\
Road.Link.0.LaneSection.0.LaneL.8.ARP = '+str(obj_id+91)+' '+str(obj_id+92)+' '+str(obj_id+93)+' '+str(obj_id+94)+' '+str(obj_id+95)+' '+str(obj_id+96)+'\n\
Road.Link.0.LaneSection.0.LaneL.9.ID = '+str(obj_id+98)+'\n\
Road.Link.0.LaneSection.0.LaneL.9 = 0 2.5 2.5 5 0 0 0\n\
Road.Link.0.LaneSection.0.LaneR.0.ID = '+str(obj_id+101)+'\n\
Road.Link.0.LaneSection.0.LaneR.0 = 0 3.5 3.5 0 0 0 0\n\
Road.Link.0.LaneSection.0.LaneR.0.ARP = '+str(obj_id+104)+' '+str(obj_id+105)+' '+str(obj_id+106)+' '+str(obj_id+107)+' '+str(obj_id+108)+' '+str(obj_id+109)+'\n\
Road.Link.0.LaneSection.0.LaneR.1.ID = '+str(obj_id+112)+'\n\
Road.Link.0.LaneSection.0.LaneR.1 = 0 3.5 3.5 0 0 0 0\n\
Road.Link.0.LaneSection.0.LaneR.1.ARP = '+str(obj_id+115)+' '+str(obj_id+116)+' '+str(obj_id+117)+' '+str(obj_id+118)+' '+str(obj_id+119)+' '+str(obj_id+120)+'\n\
Road.Link.0.LaneSection.0.LaneR.2.ID = '+str(obj_id+123)+'\n\
Road.Link.0.LaneSection.0.LaneR.2 = 0 1 1 4 0 0 0\n\
Road.Link.0.LaneSection.0.LaneR.2.ARP = '+str(obj_id+126)+' '+str(obj_id+127)+' '+str(obj_id+128)+' '+str(obj_id+129)+' '+str(obj_id+130)+' '+str(obj_id+131)+'\n\
Road.Link.0.LaneSection.0.LaneR.3.ID = '+str(obj_id+133)+'\n\
Road.Link.0.LaneSection.0.LaneR.3 = 0 2.5 2.5 5 0 0 0\n\
Road.LanePath.0 = '+str(obj_id+11)+' '+str(obj_id+2)+' 2 10 0.1 0.1\n\
Road.LanePath.1 = '+str(obj_id+22)+' '+str(obj_id+13)+' 0.25 10 0.1 0.1\n\
Road.LanePath.2 = '+str(obj_id+33)+' '+str(obj_id+24)+' 0.25 10 0.1 0.1\n\
Road.LanePath.3 = '+str(obj_id+43)+' '+str(obj_id+34)+' 0.25 10 0.1 0.1\n\
Road.LanePath.4 = '+str(obj_id+53)+' '+str(obj_id+44)+' 0.25 10 0.1 0.1\n\
Road.LanePath.5 = '+str(obj_id+64)+' '+str(obj_id+55)+' 2 10 0.1 0.1\n\
Road.LanePath.6 = '+str(obj_id+75)+' '+str(obj_id+66)+' 2 10 0.1 0.1\n\
Road.LanePath.7 = '+str(obj_id+86)+' '+str(obj_id+77)+' 2 10 0.1 0.1\n\
Road.LanePath.8 = '+str(obj_id+97)+' '+str(obj_id+88)+' 0.25 10 0.1 0.1\n\
Road.LanePath.9 = '+str(obj_id+110)+' '+str(obj_id+101)+' 2 10 0.1 0.1\n\
Road.LanePath.10 = '+str(obj_id+121)+' '+str(obj_id+112)+' 2 10 0.1 0.1\n\
Road.LanePath.11 = '+str(obj_id+132)+' '+str(obj_id+123)+' 0.25 10 0.1 0.1\n\
Route.0.ID = '+str(obj_id+137)+'\n\
Route.0.Name = Route_0\n\
Route.0.DrvPath.ID = '+str(obj_id+138)+'\n\
Route.0.DrvPath:\n\
    '+str(obj_id+110)+'\n\
Road.RL.1.RoadMarking.0.ID = '+str(obj_id+136)+' '+str(obj_id+1)+'\n\
Road.RL.1.RoadMarking.0 = 0 0 0 1 0 0 0.15 0 2 0 0 2 4 1 1 0 ""\n\
Road.RL.1.RoadMarking.0.Material.0 = 1.0,1.0,1.0 0 0 0 0 0 0 0 0 0 0 0\n\
Road.RL.1.RoadMarking.1.ID = '+str(obj_id+122)+' '+str(obj_id+112)+'\n\
Road.RL.1.RoadMarking.1 = 0 0 0 1 0 -1 0.15 0 1 0 0 2 4 1 1 0 ""\n\
Road.RL.1.RoadMarking.1.Material.0 = 1.0,1.0,1.0 0 0 0 0 0 0 0 0 0 0 0\n\
Road.RL.1.RoadMarking.2.ID = '+str(obj_id+111)+' '+str(obj_id+101)+'\n\
Road.RL.1.RoadMarking.2 = 0 0 0 1 0 -1 0.15 0 2 0 0 2 4 1 1 0 ""\n\
Road.RL.1.RoadMarking.2.Material.0 = 1.0,1.0,1.0 0 0 0 0 0 0 0 0 0 0 0\n\
Road.RL.1.RoadMarking.3.ID = '+str(obj_id+87)+' '+str(obj_id+77)+'\n\
Road.RL.1.RoadMarking.3 = 0 0 0 1 0 1 0.15 0 1 0 0 2 4 1 1 0 ""\n\
Road.RL.1.RoadMarking.3.Material.0 = 1.0,1.0,1.0 0 0 0 0 0 0 0 0 0 0 0\n\
Road.RL.1.RoadMarking.4.ID = '+str(obj_id+76)+' '+str(obj_id+66)+'\n\
Road.RL.1.RoadMarking.4 = 0 0 0 1 0 1 0.15 0 2 0 0 2 4 1 1 0 ""\n\
Road.RL.1.RoadMarking.4.Material.0 = 1.0,1.0,1.0 0 0 0 0 0 0 0 0 0 0 0\n\
Road.RL.1.RoadMarking.5.ID = '+str(obj_id+65)+' '+str(obj_id+55)+'\n\
Road.RL.1.RoadMarking.5 = 0 0 0 1 0 1 0.15 0 2 0 0 2 4 1 1 0 ""\n\
Road.RL.1.RoadMarking.5.Material.0 = 1.0,1.0,1.0 0 0 0 0 0 0 0 0 0 0 0\n\
Road.RL.1.RoadMarking.6.ID = '+str(obj_id+54)+' '+str(obj_id+44)+'\n\
Road.RL.1.RoadMarking.6 = 0 0 0 1 0 1 0.15 0 1 0 0 2 4 1 1 0 ""\n\
Road.RL.1.RoadMarking.6.Material.0 = 1.0,1.0,1.0 0 0 0 0 0 0 0 0 0 0 0\n\
Road.RL.1.RoadMarking.7.ID = '+str(obj_id+23)+' '+str(obj_id+13)+'\n\
Road.RL.1.RoadMarking.7 = 0 0 0 1 0 1 0.15 0 1 0 0 2 4 1 1 0 ""\n\
Road.RL.1.RoadMarking.7.Material.0 = 1.0,1.0,1.0 0 0 0 0 0 0 0 0 0 0 0\n\
Road.RL.1.RoadMarking.8.ID = '+str(obj_id+12)+' '+str(obj_id+2)+'\n\
Road.RL.1.RoadMarking.8 = 0 0 0 1 0 1 0.15 0 1 0 0 2 4 1 1 0 ""\n\
Road.RL.1.RoadMarking.8.Material.0 = 1.0,1.0,1.0 0 0 0 0 0 0 0 0 0 0 0\n\
Road.MaxUsedObjId = '+str(obj_id+150)+'\n\
Road.VhclStartPos = 0 0 0\n\
Road.VhclRoute = Route_0\n\
Road.RouteId = 0\n\
Env.StartTime.Year = 2019\n\
Env.StartTime.Month = 1\n\
Env.StartTime.Day = 1\n\
Env.StartTime.Hour = 12\n\
Env.StartTime.Min = 0\n\
Env.StartTime.Sec = 0\n\
Env.StartTime.DeltaUTC = 0.0\n\
Env.GNav.Active = 0\n\
Env.Temperature = 20.0\n\
Env.AirDensity = 1.205\n\
Env.AirPressure = 1.013\n\
Env.AirHumidity = 60\n\
Env.SolarRadiation = 400.0\n\
Env.RainRate = 0.0\n\
Env.VisRangeInFog = 1000.0\n\
Env.Wind.Kind = none\n\
Env.Wind.Velocity = 0.0\n\
Env.Wind.Angle = 0.0\n\
Env.Sun.Position = geographicDefinition\n\
Env.Sun.Azimuth = 180.0\n\
Env.Sun.Elevation = 45.0\n\
Env.Kind = Generic\n\
Env.Temp.Offset_Elev = -0.0065\n\
Env.Temp.Offset_sRoad.Amplify = 1.0\n\
Env.Temp.Offset_sRoad.On = 0\n\
Env.Temp.Offset_Time.Amplify = 1.0\n\
Env.Temp.Offset_Time.On = 1\n\
Env.Temp.Offset_Time:\n\
    0.0 -2.0\n\
    3.0 -2.5\n\
    6.0 -2.7\n\
    7.5 -2.7\n\
    9.0 -2.5\n\
    10.0 -2.3\n\
    11.0 -1.6\n\
    12.0 0.0\n\
    13.0 1.4\n\
    14.0 2.1\n\
    15.5 2.5\n\
    17.0 2.2\n\
    18.0 1.7\n\
    19.0 1.1\n\
    20.0 0.2\n\
    21.0 -0.6\n\
    22.0 -1.1\n\
    23.0 -1.6\n\
    24.0 -2.0\n\
Driver.ParamIdent = IPGDriver 5\n\
Driver.Mode = std\n\
Driver.Long.DrivMaxSpeed = 0\n\
Driver.Long.CruisingSpeed = 150\n\
Driver.CornerCutCoef = 0.5\n\
Driver.ConsiderTraffic = 1\n\
Driver.Traffic.TimeGapMin = 1.8\n\
Driver.Traffic.TimeGapMax = 5.0\n\
Driver.Traffic.DistMin = 6\n\
Driver.Traffic.DistMax = 250\n\
Driver.Traffic.EcoCoef = 0.75\n\
Driver.Traffic.Overtake = 0\n\
Driver.Traffic.Overtake_Rate = 1\n\
Driver.Traffic.Overtake_dSpeedMin = 10\n\
Driver.Long.dtAccBrake = 0.5\n\
Driver.Long.axMax = 3.0\n\
Driver.Long.axMin = -4.0\n\
Driver.Long.ayMax = 4.0\n\
Driver.Long.GGExp:\n\
    50 1.0 1.0\n\
Driver.Long.DevMax = 0.0\n\
Driver.Long.tReact = 0.0\n\
Driver.Long.TractionControl = 1\n\
Driver.DecShift.UseBrakePark = 0\n\
Driver.DecShift.tSwitchGear = 1.0\n\
Driver.DecShift.nEngine.Limits:\n\
    1500 4000\n\
Driver.DecShift.nEngine.Shift:\n\
    2000 3000\n\
Driver.Lat.DevMax = 0.0\n\
Driver.Lat.tReact = 0.0\n\
Driver.Knowl.Long.tActionMin = 4\n\
Driver.Knowl.Lat.StWhlAngleMax = 630\n\
Driver.Knowl.Lat.StWhlAngleVelMax = 500\n\
Driver.Knowl.Lat.StWhlAngleAccMax = 3000\n\
Driver.Learn.VehicleLimits.TestRun =\n\
Driver.Learn.VehicleLimits.Date = 0\n\
Driver.Learn.ControllerDyn.TestRun =\n\
Driver.Learn.ControllerDyn.Date = 0\n\
Driver.Learn.MaxSpeed.TestRun =\n\
Driver.Learn.MaxSpeed.Date = 0\n\
Driver.Learn.Remember = 0\n\
Driver.Learn.Friction = 1.0\n\
Driver.Knowl.Long.tPreviewBra = 0.6\n\
Driver.Knowl.Long.tPreviewAcc = 1.5\n\
Driver.Knowl.Lat.tPreview = 0.8\n\
Driver.Learn.NEng_S = 1\n')


    with open(TESTRUN_PATH, 'w') as f:
        for item in content_lines:
            f.write("%s" % item)

    print(f'Saved files to: {TESTRUN_PATH}')
    
    
def outputvideo(scene_num):
    import os
    from shutil import copyfile

    ## optional create video from camera data
    #temp_path = LYFT_DATASET_ROOT+'/temp'
    #if not os.path.exists(temp_path):
    #    os.mkdir(temp_path)

    img_list = []
    scene = level5data.scene[scene_num]
    scene_token = scene['token']
    sample_token = scene['first_sample_token']
    counter = 0
    while sample_token is not '':
        sample = level5data.get('sample', sample_token)
        sample_data = level5data.get('sample_data', sample['data']['CAM_FRONT'])
        img_list.append('file '+LYFT_DATASET_ROOT+sample_data['filename'])
        #copyfile(LYFT_DATASET_ROOT+sample_data['filename'],temp_path+'/img_'+str(counter)+'.jpeg')
        counter += 1
        sample_token = sample['next']


    img_list_path = LYFT_DATASET_ROOT+'/img_list.txt'
    video_path = LYFT_DATASET_ROOT+'CarMaker/video-'+str(scene_num)+'.mp4'
    with open(img_list_path, 'w') as f:
        for item in img_list:
            f.write("%s\n" % item)

    !ffmpeg -f concat -r 5 -safe 0 -i $img_list_path -r 5 -c:v libx264 -pix_fmt yuv420p $video_path

    print('Video saved to: '+video_path)


In [4]:
## load tracking data

def loadDetectionDict(scene_num,config):
    import json

    #scene: index of scene
    #config: small,middle,large

    det_dict_path = '/home/itiv/Desktop/lyft-kaggle-dataset/train/CarMaker/gt_data/gt_data_scene-'+str(scene_num)+'-'+config+'.json'
    det_dict = json.load(open(det_dict_path))
    return det_dict



In [5]:

def ann_in_boundries(ego,ann,config,plot):
    from pyquaternion import Quaternion
    import matplotlib.pyplot as plt

    
    
    ###TODO!!!!!!!!!!
    #detection boundries TODO filter ground truth out of detection boundary!!!
    det_bound = [50,50,50,50] #dist to front,right,back,left
    if model == 'middle':
        det_bound = [100,70,70,70]
    elif model == 'large':
        det_bound  = [150,100,100,100]
    elif model == 'smallv2':
        det_bound  = [50,10,10,10]
    elif model == 'middlev2':
        det_bound  = [100,10,10,10]
    elif model == 'largev2':
        det_bound  = [150,10,10,10]

    width = det_bound[1]+det_bound[3]
    length = det_bound[0]+det_bound[2]
    
    center_dist = det_bound[0]-length/2 
    
    ego_yaw = Quaternion(ego['rotation']).yaw_pitch_roll[0]
    ego_x_center = ego['translation'][0]
    ego_y_center = ego['translation'][1]
    
    boundary_box_center_x = np.cos(ego_yaw)*center_dist+ego_x_center
    boundary_box_center_y = np.sin(ego_yaw)*center_dist+ego_y_center
    
        

    traffic_yaw = Quaternion(ann['rotation']).yaw_pitch_roll[0]
    traffic_x_center = ann['translation'][0]
    traffic_y_center = ann['translation'][1]
    traffic_width = ann['size'][0]
    traffic_height = ann['size'][1]
    
    ann_rect = []
    ego_rect = []
    
    ann_rect.append([ann['translation'][0],ann['translation'][1],ann['size'][1],ann['size'][0],traffic_yaw])
    ego_rect.append([boundary_box_center_x,boundary_box_center_y,length,width,ego_yaw])
    
    
    ann_rect = np.array(ann_rect)
    ego_rect = np.array(ego_rect)

    ego_pos = rect_polygon(ego_x_center,ego_y_center,traffic_height,traffic_width,ego_yaw)
    
    
    iou = intersection_over_union(ann_rect,ego_rect,plot)
    
    print(iou)
    
    return (iou[0,0]>0)



def load_ground_truth_data(scene_num,config):
    from pyquaternion import Quaternion
    import operator
    import math



    scene = level5data.scene[scene_num]
    scene_token = scene['token']

    sample_token = scene['first_sample_token']
    sample = level5data.get('sample', sample_token)
    sample_data = level5data.get('sample_data', sample['data']['LIDAR_TOP'])
    ego_pose = level5data.get('ego_pose',sample_data['ego_pose_token'])


    trans_offset = ego_pose['translation']


    gt_dict = {'samples':[]}
    plot = True

    for index in range(scene['nbr_samples']):
        sample = level5data.get('sample', sample_token)

        if(math.fmod(index,100) == 0):
            plot = True
            

        sample_token = sample['next']
        
        #if sample_token == '':
         #   print('Finished earlier at: '+str(index))
        
        sample_dict = {'token':sample_token,'timestamp':sample['timestamp'],'anns':[]}

        sample_data = level5data.get('sample_data', sample['data']['LIDAR_TOP'])
        ego_pose = level5data.get('ego_pose',sample_data['ego_pose_token'])
        for ann_token in sample['anns']:
            ann = level5data.get('sample_annotation', ann_token)
            
            if ann_in_boundries(ego_pose,ann,config,plot) and (ann['category_name'] == 'car' or ann['category_name'] == 'bus' or ann['category_name'] == 'construction_vehicle' or ann['category_name'] == 'motorcycle' or ann['category_name'] == 'truck' or ann['category_name'] == 'trailer'):     
                yaw = Quaternion(ann['rotation']).yaw_pitch_roll[0]
                ann_dict = {'instance':ann['instance_token'],'translation':list(map(operator.sub,ann['translation'],trans_offset)),'size':ann['size'],'rotation':yaw}
                sample_dict['anns'].append(ann_dict)

        plot = False
        gt_dict['samples'].append(sample_dict)
        
        
    return gt_dict
    
    


In [6]:
## https://arxiv.org/pdf/1603.00831.pdf
#
#
#Multiple Object Tracking Accuracy
# ratio of misses in the sequence, computed over the total number of objects present in all frames
# ratio of false positives (bounding box with iou < T)
# ratio of mismatches (id change)




def rect_polygon(x, y, width, height, angle):
    from shapely.geometry import Polygon
    from shapely.affinity import rotate, translate
    import math

    """Return a shapely Polygon describing the rectangle with centre at
    (x, y) and the given width and height, rotated by angle quarter-turns.

    """
    w = width / 2
    h = height / 2
    p = Polygon([(-w, -h), (w, -h), (w, h), (-w, h)])
    return translate(rotate(p, math.degrees(angle)), x, y)


def intersection_over_union(rects_a, rects_b,plot=False):
    import rtree.index
    from shapely.affinity import rotate, translate
    from shapely.geometry import Polygon
    import numpy as np
    import matplotlib.pyplot as plt
    import math

    """Calculate the intersection-over-union for every pair of rectangles
    in the two arrays.

    Arguments:
    rects_a: array_like, shape=(M, 5)
    rects_b: array_like, shape=(N, 5)
        Rotated rectangles, represented as (centre x, centre y, width,
        height, rotation in quarter-turns).

    Returns:
    iou: array, shape=(M, N)
        Array whose element i, j is the intersection-over-union
        measure for rects_a[i] and rects_b[j].

    """
    m = len(rects_a)
    n = len(rects_b)
    if m > n:
        # More memory-efficient to compute it the other way round and
        # transpose.
        return intersection_over_union(rects_b, rects_a,plot).T

    # Convert rects_a to shapely Polygon objects.
    polys_a = [rect_polygon(*r) for r in rects_a]

    # Build a spatial index for rects_a.
    index_a = rtree.index.Index()
    for i, a in enumerate(polys_a):
        index_a.insert(i, a.bounds)
    
    #print((index_a))
        
    # Find candidate intersections using the spatial index.
    iou = np.zeros((m, n))
    for j, rect_b in enumerate(rects_b):
        b = rect_polygon(*rect_b)
        if plot:
            x,y = b.exterior.xy
            if x[0] < 500 and y[0] < 500:
                plt.plot(x,y,'g-')
        for i in range(len(polys_a)):#index_a.intersection(b.bounds):
            a = polys_a[i]
            if(j == 1 and plot):
                x,y = a.exterior.xy
                if x[0] < 500 and y[0] < 500:
                    plt.plot(x,y,'r-')
        
            intersection_area = a.intersection(b).area
            if intersection_area:
                iou[i, j] = intersection_area / a.union(b).area
    
    if plot:
        plt.show()
        print(iou)
    return iou



In [7]:


def get_iou_for_scene(gt_dict,det_dict,id_counter_init):
    
    import numpy as np
    import matplotlib.pyplot as plt
    
    iou_arr = []
    gt_obj_id = []
    det_obj_id = []

    global id_counter
    id_counter = id_counter_init
    global id_conversion_dict
    id_conversion_dict = {}


    def instance_token_to_id(token):
        global id_counter
        global id_conversion_dict
        if not token in id_conversion_dict:
            id_counter += 1
            id_conversion_dict.update( {token : id_counter} )

        return id_conversion_dict[token]




    for sample_index in range(len(gt_dict['samples'])):

        gt_rects = []
        gt_ids = []
        for anns in gt_dict['samples'][sample_index]['anns']:
            rect = [anns['translation'][0],anns['translation'][1],anns['size'][1],anns['size'][0],anns['rotation']]
            gt_rects.append(rect)
            gt_ids.append(instance_token_to_id(anns['instance']))

        det_rects = []
        det_ids = []
        for anns in det_dict['samples'][sample_index]['anns']:
            rect = [anns['translation'][0],anns['translation'][1],anns['size'][1],anns['size'][0],anns['rotation']]
            det_rects.append(rect)
            det_ids.append(anns['instance'])


        gt_obj_id.append(gt_ids)
        det_obj_id.append(det_ids)


        gt_rects = np.array(gt_rects)
        det_rects = np.array(det_rects)


        iou_arr.append(intersection_over_union(gt_rects,det_rects,id_counter_init == 0))

        ### returns 
        #  det det det det det
        #gt
        #gt
        #gt
        #gt
        #gt
        #print(intersection_over_union(gt_rects,det_rects,False))
        #print(len(gt_rects))
        #print(len(det_rects))



    return iou_arr,gt_obj_id,det_obj_id,id_counter


    '''

    '''


In [9]:
## sum of gt
## sum of misses
## sum of false positive
## sum of mismatch

## dist = 1 - iou

IOU_TRESHOLD = 0.5


def get_MOTA_Acc(iou_arr,gt_obj_id,det_obj_id):
    import motmetrics as mm
    import numpy as np

    # Create an accumulator that will be updated during each frame
    acc = mm.MOTAccumulator(auto_id=True)

    for index,iou in enumerate(iou_arr):

        
        #dist = np.subtract(np.ones(iou.shape),iou)
        dist = iou
        
        

        for i, r in enumerate(dist):
            for j, c in enumerate(r):
                if dist[i,j] > (1-IOU_TRESHOLD):
                    dist[i,j] = np.nan

        # Call update once for per frame. For now, assume distances between
        # frame objects / hypotheses are given.
        acc.update(
            gt_obj_id[index],                     # Ground truth objects in this frame
            det_obj_id[index],                  # Detector hypotheses in this frame
            dist.tolist()
        )
    return acc



In [9]:
#### get test scenes!
# Load the SDK
%matplotlib inline
from lyft_dataset_sdk.lyftdataset import LyftDataset

# Load the dataset
# Adjust the dataroot parameter below to point to your local dataset path.
# The correct dataset path contains at least the following four folders (or similar): images, lidar, maps, v1.0.1-train
level5datalyft = LyftDataset(data_path='/home/itiv/Desktop/lyft-dataset/', json_path='/home/itiv/Desktop/lyft-dataset/'+'/v1.02-train', verbose=True)

## extract test scenes...
counter = 0
test_scene_index = []
for kaggleindex,kagglescene in enumerate(level5data.scene):
    for index,lyftscene in enumerate(level5datalyft.scene):
        if kagglescene['name'] == lyftscene['name']:
            counter += 1
            break
        elif index == len(level5datalyft.scene)-1:
            #not found:
            counter += 1
            test_scene_index.append(kaggleindex)


test_scene_index

9 category,
18 attribute,
4 visibility,
15991 instance,
8 sensor,
128 calibrated_sensor,
149072 ego_pose,
148 log,
148 scene,
18634 sample,
149072 sample_data,
539765 sample_annotation,
1 map,
Done loading in 6.4 seconds.
Reverse indexing ...
Done reverse indexing in 1.9 seconds.


[0,
 4,
 6,
 12,
 25,
 30,
 32,
 41,
 45,
 46,
 48,
 61,
 70,
 76,
 79,
 80,
 83,
 84,
 87,
 104,
 108,
 114,
 120,
 130,
 134,
 141,
 142,
 146,
 153,
 174,
 175,
 176]

In [42]:


index = 3

scene = level5data.scene[test_scene_index[index]]
scene_token = scene['token']

print(test_scene_index[index])

level5data.render_scene(scene_token)

12


In [1]:
## run this
    

    
    
    
import motmetrics as mm
import numpy as np  
from datetime import datetime

#models = ['middle','middle','large','large','large']
#nets = ['/model_middle_range/voxelnet-550000.tckpt','/model_middle_range/voxelnet-890670.tckpt','/model_large_range/voxelnet-250000.tckpt','/model_large_range/voxelnet-500000.tckpt','/model_large_range/voxelnet-890670.tckpt']

#models = ['large','large','large','small','small']
#nets = ['/model_large_range/voxelnet-250000.tckpt','/model_large_range/voxelnet-500000.tckpt','/model_large_range/voxelnet-890670.tckpt','/model_small_range/voxelnet-259585.tckpt','/model_small_range/voxelnet-121598.tckpt']

models = ['small']
nets = ['/model_small_range/voxelnet-259585.tckpt']


skip_scenes = [0,46]

eval_scenes = [30]

##create pkl    
#for index,model in enumerate(models):
#    for scene_num in test_scene_index:
#   

    #outputvideo(scene_num)

print('###########eval#############')        
for index,model in enumerate(models):
    iou_arr = []
    gt_ids = []
    det_ids = []
    id_counter = 0
    accs = []
    for scene_num in test_scene_index:
        
        if scene_num in []:
        
        
    #try:
        #create_pkl(scene_num)
        print(scene_num)
        set_config_file(scene_num,model)
        track(scene_num,model,nets[index])

        det_dict = loadDetectionDict(scene_num,model)
        gt_dict = load_ground_truth_data(scene_num,model)
        scene_iou_arr,scene_gt_ids,scene_det_ids,id_counter_val = get_iou_for_scene(gt_dict,det_dict,id_counter)
        id_counter = id_counter_val
        #accs.append(get_MOTA_Acc(scene_iou_arr,scene_gt_ids,scene_det_ids))
        det_ids.extend(scene_det_ids)
        gt_ids.extend(scene_gt_ids)
        iou_arr.extend(scene_iou_arr)
        print(f'finished analyzing scene: {scene_num}')
    #except:
        print('#################### ERROR at '+str(scene_num)+' ########################')

        
    acc = get_MOTA_Acc(iou_arr,gt_ids,det_ids)

    print('creating metric')
    
    mh = mm.metrics.create()
    summary = mh.compute(acc, metrics=mm.metrics.motchallenge_metrics, name=model)
    print(summary)
    
    now = datetime.now() # current date and time
    date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
    
    with open('EvaluationResults.txt', "a") as resultFile:
        resultFile.write("\n #### "+str(date_time)+' ### \n')
        resultFile.write("Net: "+nets[index]+' \n')

    
    summary.to_csv('EvaluationResults.txt', sep='\t', mode='a',index=True)

    
    

###########eval#############


NameError: name 'test_scene_index' is not defined

In [10]:
    
import motmetrics as mm
import numpy as np  
from datetime import datetime

#models = ['middle','middle','large','large','large']
#nets = ['/model_middle_range/voxelnet-550000.tckpt','/model_middle_range/voxelnet-890670.tckpt','/model_large_range/voxelnet-250000.tckpt','/model_large_range/voxelnet-500000.tckpt','/model_large_range/voxelnet-890670.tckpt']

#models = ['large','large','large','small','small']
#nets = ['/model_large_range/voxelnet-250000.tckpt','/model_large_range/voxelnet-500000.tckpt','/model_large_range/voxelnet-890670.tckpt','/model_small_range/voxelnet-259585.tckpt','/model_small_range/voxelnet-121598.tckpt']

models = ['small']
nets = ['/model_small_range/voxelnet-259585.tckpt']


skip_scenes = [0,46]

eval_scenes = [30]

##create pkl    
#for index,model in enumerate(models):
#    for scene_num in test_scene_index:
#   

    #outputvideo(scene_num)

print('###########eval#############')        
for index,model in enumerate(models):
    iou_arr = []
    gt_ids = []
    det_ids = []
    id_counter = 0
    accs = []
    for scene_num in eval_scenes:
        #create_pkl(scene_num)
        print(scene_num)
        set_config_file(scene_num,model)
        track(scene_num,model,nets[index])

        det_dict = loadDetectionDict(scene_num,model)
        gt_dict = load_ground_truth_data(scene_num,model)
        scene_iou_arr,scene_gt_ids,scene_det_ids,id_counter_val = get_iou_for_scene(gt_dict,det_dict,id_counter)
        id_counter = id_counter_val
        #accs.append(get_MOTA_Acc(scene_iou_arr,scene_gt_ids,scene_det_ids))
        det_ids.extend(scene_det_ids)
        gt_ids.extend(scene_gt_ids)
        iou_arr.extend(scene_iou_arr)
        print(f'finished analyzing scene: {scene_num}')
        print('#################### ERROR at '+str(scene_num)+' ########################')

        
    acc = get_MOTA_Acc(iou_arr,gt_ids,det_ids)

    print('creating metric')
    
    mh = mm.metrics.create()
    summary = mh.compute(acc, metrics=mm.metrics.motchallenge_metrics, name=model)
    print(summary)
    
    now = datetime.now() # current date and time
    date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
    
    with open('EvaluationResults.txt', "a") as resultFile:
        resultFile.write("\n #### "+str(date_time)+' ### single scene analysis \n')
        resultFile.write("Net: "+nets[index]+' \n')

    
    summary.to_csv('EvaluationResults.txt', sep='\t', mode='a',index=True)

###########eval#############
30
Paths set in config file
Detecting...this might take several minutes...
/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_NVVM=/usr/local/cuda/nvvm/lib64/libnvvm.so.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_LIBDEVICE=/usr/local/cuda/nvvm/libdevice.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/home/itiv/ana

KeyboardInterrupt: 

In [ ]:
##### automated Training and testing....
import numpy as np
from second.protos import pipeline_pb2
from google.protobuf import text_format



models = ['smallv2','middlev2','largev2']
step_size = 200000
eval_scenes = [30,104,108,130,141,142]

for epoch in range(4):
    for model in models:
        try:
            LYFT_TRAINVAL_DATASET_ROOT = '/home/itiv/Desktop/lyft-dataset/'
            config_path = 'Lyft-Detector/second.pytorch/second/configs/nuscenes/all.pp.lowa_'+model+'_range.config'
            model_path = 'Lyft-Detector/second.pytorch/second/model/model_'+model+'_range'


            ## set paths in config file...

            step = step_size*(epoch+1)


            config = pipeline_pb2.TrainEvalPipelineConfig()
            with open(config_path, "r") as f:
                proto_str = f.read()
                text_format.Merge(proto_str, config)
            config.train_input_reader.dataset.kitti_info_path = LYFT_TRAINVAL_DATASET_ROOT+"/infos_train.pkl"
            config.train_input_reader.dataset.kitti_root_path = LYFT_TRAINVAL_DATASET_ROOT

            config.train_input_reader.preprocess.database_sampler.database_info_path = LYFT_TRAINVAL_DATASET_ROOT+"/infos_train.pkl"

            config.eval_input_reader.dataset.kitti_info_path = LYFT_TRAINVAL_DATASET_ROOT+"/infos_val.pkl"
            config.eval_input_reader.dataset.kitti_root_path = LYFT_TRAINVAL_DATASET_ROOT
            config.train_config.steps = step

            config_text = text_format.MessageToString(config)
            with open(config_path, "w") as f:
                f.write(config_text)

            print('Paths set in config file')


            ##start training

            !python ./Lyft-Detector/second.pytorch/second/pytorch/train.py train --config_path=$config_path --model_dir=$model_path


            net = '/model_'+model+'_range/voxelnet-'+str(step)+'.tckpt'


            ##evaluate
            iou_arr = []
            gt_ids = []
            det_ids = []
            id_counter = 0
            accs = []
            for scene_num in eval_scenes:
                #create_pkl(scene_num)
                print(scene_num)
                set_config_file(scene_num,model)
                track(scene_num,model,net)

                det_dict = loadDetectionDict(scene_num,model)
                gt_dict = load_ground_truth_data(scene_num,model)
                scene_iou_arr,scene_gt_ids,scene_det_ids,id_counter_val = get_iou_for_scene(gt_dict,det_dict,id_counter)
                id_counter = id_counter_val
                det_ids.extend(scene_det_ids)
                gt_ids.extend(scene_gt_ids)
                iou_arr.extend(scene_iou_arr)
                print(f'finished analyzing scene: {scene_num}')


            acc = get_MOTA_Acc(iou_arr,gt_ids,det_ids)

            print('creating metric')

            mh = mm.metrics.create()
            summary = mh.compute(acc, metrics=mm.metrics.motchallenge_metrics, name=model)
            print(summary)

            now = datetime.now() # current date and time
            date_time = now.strftime("%m/%d/%Y, %H:%M:%S")

            with open('EvaluationResults.txt', "a") as resultFile:
                resultFile.write("\n #### "+str(date_time)+' ### single scene analysis \n')
                resultFile.write("Net: "+net+' \n')


            summary.to_csv('EvaluationResults.txt', sep='\t', mode='a',index=True)
        except Exception as e: 
            print(e)
            print('Error....skipping')   
        
        
        

models = ['small','middle','large']


for epoch in range(4):
    for model in models:
        try:
            LYFT_TRAINVAL_DATASET_ROOT = '/home/itiv/Desktop/lyft-dataset/'
            config_path = 'Lyft-Detector/second.pytorch/second/configs/nuscenes/all.pp.lowa_'+model+'_range.config'
            model_path = 'Lyft-Detector/second.pytorch/second/model/model_'+model+'_range'


            ## set paths in config file...

            step = step_size*(epoch+1)


            config = pipeline_pb2.TrainEvalPipelineConfig()
            with open(config_path, "r") as f:
                proto_str = f.read()
                text_format.Merge(proto_str, config)
            config.train_input_reader.dataset.kitti_info_path = LYFT_TRAINVAL_DATASET_ROOT+"/infos_train.pkl"
            config.train_input_reader.dataset.kitti_root_path = LYFT_TRAINVAL_DATASET_ROOT

            config.train_input_reader.preprocess.database_sampler.database_info_path = LYFT_TRAINVAL_DATASET_ROOT+"/infos_train.pkl"

            config.eval_input_reader.dataset.kitti_info_path = LYFT_TRAINVAL_DATASET_ROOT+"/infos_val.pkl"
            config.eval_input_reader.dataset.kitti_root_path = LYFT_TRAINVAL_DATASET_ROOT
            config.train_config.steps = step

            config_text = text_format.MessageToString(config)
            with open(config_path, "w") as f:
                f.write(config_text)

            print('Paths set in config file')


            ##start training

            !python ./Lyft-Detector/second.pytorch/second/pytorch/train.py train --config_path=$config_path --model_dir=$model_path


            net = '/model_'+model+'_range/voxelnet-'+str(step)+'.tckpt'


            ##evaluate
            iou_arr = []
            gt_ids = []
            det_ids = []
            id_counter = 0
            accs = []
            for scene_num in eval_scenes:
                #create_pkl(scene_num)
                print(scene_num)
                set_config_file(scene_num,model)
                track(scene_num,model,net)

                det_dict = loadDetectionDict(scene_num,model)
                gt_dict = load_ground_truth_data(scene_num,model)
                scene_iou_arr,scene_gt_ids,scene_det_ids,id_counter_val = get_iou_for_scene(gt_dict,det_dict,id_counter)
                id_counter = id_counter_val
                det_ids.extend(scene_det_ids)
                gt_ids.extend(scene_gt_ids)
                iou_arr.extend(scene_iou_arr)
                print(f'finished analyzing scene: {scene_num}')


            print('acc')
            acc = get_MOTA_Acc(iou_arr,gt_ids,det_ids)

            print('creating metric')

            mh = mm.metrics.create()
            summary = mh.compute(acc, metrics=mm.metrics.motchallenge_metrics, name=model)
            print(summary)

            now = datetime.now() # current date and time
            date_time = now.strftime("%m/%d/%Y, %H:%M:%S")

            with open('EvaluationResults.txt', "a") as resultFile:
                resultFile.write("\n #### "+str(date_time)+' ### single scene analysis \n')
                resultFile.write("Net: "+net+' \n')


            summary.to_csv('EvaluationResults.txt', sep='\t', mode='a',index=True)
        except Exception as e: 
            print(e)
            print('Error....skipping')        
        


Paths set in config file
/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_NVVM=/usr/local/cuda/nvvm/lib64/libnvvm.so.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/numba/cuda/envvars.py:17: NumbaWarning: 
Environment variables with the 'NUMBAPRO' prefix are deprecated and consequently ignored, found use of NUMBAPRO_LIBDEVICE=/usr/local/cuda/nvvm/libdevice.

For more information about alternatives visit: ('http://numba.pydata.org/numba-doc/latest/cuda/overview.html', '#cudatoolkit-lookup')
  warnings.warn(errors.NumbaWarning(msg))
/home/itiv/anaconda3/envs/lyft_detector/lib/python3.6/site-packages/numba/cuda/envvars.py:17:

WORKER 0 seed: 1584616813
WORKER 1 seed: 1584616814
WORKER 2 seed: 1584616815

runtime.step=50, runtime.steptime=0.2582, runtime.voxel_gene_time=0.007626, runtime.prep_time=0.01445, loss.cls_loss=416.1, loss.cls_loss_rt=6.691, loss.loc_loss=2.033, loss.loc_loss_rt=1.86, loss.loc_elem=[0.2136, 0.05881, 0.2341, 0.07719, 0.1219, 0.1151, 0.1096], loss.cls_pos_rt=5.452, loss.cls_neg_rt=1.239, loss.dir_rt=0.5552, rpn_acc=0.7726, misc.num_vox=44326, misc.num_pos=9, misc.num_neg=5089, misc.num_anchors=5100, misc.lr=0.0003, misc.mem_usage=45.2

runtime.step=100, runtime.steptime=0.1821, runtime.voxel_gene_time=0.008033, runtime.prep_time=0.0149, loss.cls_loss=208.7, loss.cls_loss_rt=0.6953, loss.loc_loss=1.758, loss.loc_loss_rt=1.658, loss.loc_elem=[0.1408, 0.04813, 0.2803, 0.0541, 0.04867, 0.08038, 0.1764], loss.cls_pos_rt=0.1889, loss.cls_neg_rt=0.5064, loss.dir_rt=0.6299, rpn_acc=0.8856, misc.num_vox=45443, misc.num_pos=10, misc.num_neg=5088, misc.num_anchors=5100, misc.lr=0.0003, misc.mem_u